<a href="https://colab.research.google.com/github/Kristelwen/TM10007_PROJECT/blob/master/begin_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment template

In [9]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [0]:
# Importing modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [11]:
# Data loading functions. Uncomment the one you want to use
# from adni.load_data import load_data
from brats.load_data import load_data
#from hn.load_data import load_data
# hoi
# en als ik dit type?

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 167
The number of columns: 725


In [0]:
data_top = data.head()  

# display  
# data_top # = show data_top



---


# Outline opdracht


---


Gebaseerd op beeldvorming moeten we eruit kunnen halen wat voor een soort tumor het is?
Je wil weten of het HIGH of LOW grade is.

In [0]:
# Preprocessing the data 

# Split data and labels
labels = data['label']
data2 = data.drop(columns="label")  # Data without labels --> kolom labels is weggehaald

# Convert labels 'GBM' and 'LGG' to respectively 0 and 1 --> BGM = 0, LGG = 1
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Delete columns with strings (2 columns)
data_strings = data2.select_dtypes(include=[object])
columns_strings = list(data_strings.columns)
data_no_strings = data2.drop(columns_strings, axis=1)

# Replace infinity values with NaN
data_no_strings = data_no_strings.replace([np.inf, -np.inf], np.nan)
# print(np.isinf(data_no_strings.values).any()) - check if dataframe contains infinity values

# Drop columns which contain NaN values
data_drop = data_no_strings.dropna(axis=1)

# Feature scaling
standard_scaler = StandardScaler()
data_scaled = standard_scaler.fit_transform(data_drop.values)
data_df = pd.DataFrame(data_scaled, index = data_drop.index, columns = data_drop.columns)

# Split the data in a train (80%) and test set (20%) --> evt aanpassing dmv cross-validation
data_train, data_test, label_train, label_test = train_test_split(data_df, labels, test_size=0.2)

# PCA
pca_train = PCA(n_components=20)  # Create a PCA with 20 components --> willen we er 20??
pca_train.fit(data_train)  # Fit PCA
data_train_pca = pca_train.transform(data_train)  # Transform train data using PCA
df_train_pca = pd.DataFrame(data_train_pca, index = data_train.index)  # Put train data back in dataframe with 20 most important features
# print(df_train_pca)

data_test_pca = pca_train.transform(data_test)  # Transform test data using PCA


Vanaf hier begin van SVM. 

In [16]:
!pip install SimpleITK 
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import SimpleITK as sitk

# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel

In [27]:
# Construct classifiers
svmlin = SVC(kernel='linear', gamma='scale')  # linear kernel
svmrbf = SVC(kernel='rbf', gamma='scale')  # radial basis function kernel
svmpoly = SVC(kernel='poly', degree=3, gamma='scale')  # polynomial kernel

clsfs = [svmlin, svmpoly, svmrbf] # verschillende classifiers

# svmlin
svmlin.fit(data_train_pca, label_train) # fit classifier op trainingsset
test = svmlin.predict(data_test_pca) # test classifier op testset
error = sum(abs(test - label_test)) # hoevaak komt het niet overeen met het gegven label?
print(f'Misclassified with linear kernel: {error}/{len(test)}')

# svmrbf
svmrbf.fit(data_train_pca, label_train) # fit classifier op trainingsset
test = svmrbf.predict(data_test_pca) # test classifier op testset
error = sum(abs(test - label_test)) # hoevaak komt het niet overeen met het gegven label?
print(f'Misclassified with rbf kernel: {error}/{len(test)}')

# svmpoly
svmpoly.fit(data_train_pca, label_train) # fit classifier op trainingsset
test = svmpoly.predict(data_test_pca) # test classifier op testset
error = sum(abs(test - label_test)) # hoevaak komt het niet overeen met het gegven label?
print(f'Misclassified with poly kernel: {error}/{len(test)}')


Misclassified with linear kernel: 3/34
Misclassified with rbf kernel: 3/34
Misclassified with poly kernel: 4/34
